
PyTorch는 신경망을 생성하고 학습시키는 것을 도와주기 위해서 
torch.nn, torch.optim, Dataset 그리고 DataLoader와 같은 모듈과 클래스를 제공.

가중치에 대해서는 `requires_grad` 를 초기화(initialization) **다음에** 설정
> 해당 단계가 기울기에 포함되는 것을 원치 않기 때문

(PyTorch에서 `_` 다음에 오는 메소드 이름은 연산이 인플레이스(in-place)로 수행되는 것을 의미)

`preds` 텐서(tensor)는 텐서 값 외에도, 또한 기울기 함수(gradient function)를 담고 있습니다.


### `torch.nn.functional` 사용하기
- 직접 작성한 활성화, 손실 함수를 torch.nn.functional 의 함수로 대체
- 일반적으로 F 네임스페이스를 통해 import 
- 다양한 손실 및 활성화 함수 뿐만 아니라, pooling 함수와 같이 신경망을 만드는데 편리한 몇 가지 함수도 찾을 수 있음
- 만약 음의 로그 우도 손실과 로그 소프트맥스 활성화 함수 사용시 이 둘을 결함한 단일 함수인 F.cross_entropy를 제공


## nn.Module 을 이용하여 리팩토링
- 더 명확하고 간결한 훈련 루프를 위해 nn.Module 및 nn.Parameter 사용
- nn.Module (자체가 클래스이고 상태를 추적할 수 있는) 하위 클래스 (subclass) 생성
- 이 경우에는 포워드(forward) 단계에 대한 가중치, 절편, 그리고 메소드 등을 유지하는 클래스를 만듦
- nn.Module은 사용할 몇 가지 속성과 메소드를 가지고 있음

```
`nn.Module` (대문자 M) 은 PyTorch 의 특정 개념이고, 우리는 이 클래스를 많이 사용할 것입니다. 
`nn.Module` 를 Python 의 코드를 임포트하기 위한 코드 파일인 [module] (소문자 `m`) 의 개념과 헷갈리지 말아주세요.
```



### nn.Linear를 사용하여 리팩토링
- `self.weights` 및 `self.bias` 를 수동으로 정의 및 초기화
- `xb  @ self.weights + self.bias` 를 계산하는 대신에, 위의 모든 것을 해줄 PyTorch 클래스인 [nn.Linear]를 선형 레이어로 사용
-  PyTorch 에는 다양한 유형의 코드를 크게 단순화 할 수 있는 미리 정의된 레이어가 있고 이는 또한 종종 기존 코드보다 속도를 빠르게 함


### torch.optim 을 이용하여 리팩토링
- 각 매개변수를 수동으로 업데이트 하는 대신, optimizer의 step 메소드를 사용하여 업데이트 진행
```
with torch.no_grad():
    for p in model.parameters(): p -= p.grad * lr
    model.zero_grad()
```
를 아래처럼 변경
```
opt.step()
opt.zero_grad()
```


### Dataset 을 이용하여 리팩토링하기
- PyTorch 에는 추상 Dataset 클래스가 있음
- Dataset 은 `__len__` 함수 (Python의 표준 `len` 함수에 의해 호출됨) 및 `__getitem__` 함수를 가진 어떤 것이라도 될 수 있음
- 이 함수들을 인덱싱(indexing)하기 위한 방법으로 사용
- TensorDataset은 텐서를 감싸는 Dataset


### `DataLoader` 를 사용하여 리팩토링하기
- PyTorch 의 `DataLoader` 는 배치 관리를 담당 
- 모든 `Dataset` 으로부터 `DataLoader` 를 생성할 수 있음
- `DataLoader` 는 배치들에 대해서 반복하기 쉽게 만들어줌
- `train_ds[i*bs : i*bs+bs]` 를 사용하는 대신, `DataLoader` 는 매 미니배치를 자동적으로 제공


### 검증(validation) 추가하기
- 훈련 데이터를 섞는(shuffling) 것은 배치와 과적합 사이의 상관관계를 방지
- 반면, 검증 손실(validation loss)은 검증 데이터셋을 섞든 안섞든 동일
- 검증 데이터셋에 대한 배치 크기는 학습 데이터셋 배치 크기의 2배 사용
- 검증 데이터셋에 대해서는 역전파가 필요하지 않으므로 메모리 사용량 적음


### fit() 와 get_data() 생성하기
- `fit` 은 모델을 훈련하고 각 에폭에 대한 훈련 및 검증 손실을 계산하는 작업을 수행
- `get_data` 는 학습 및 검증 데이터셋에 대한 dataloader 를 출력


### CNN 으로 넘어가기
- 3개의 컨볼루션 레이어로 신경망을 구축
- Pytorch의 사전정의된 Conv2d 클래스를 컨볼루션 레이어로 사용
- 3개의 컨볼루션 레이어로 CNN을 정의
- 각 컨볼루션 뒤에는 ReLU가 있음
- 마지막으로 평균 풀링(average pooling)을 수행
- `view` 는 PyTorch의 NumPy `reshape` 버전
- 모멘텀(Momentum) 은 이전 업데이트도 고려하고 일반적으로 더 빠른 훈련으로 이어지는 확률적 경사 하강법의 변형


### `nn.Sequential` 사용하기
`torch.nn` 에는 코드를 간단히 사용할 수 있는 또 다른 편리한 클래스인 Sequential 존재
`Sequential` 객체는 그 안에 포함된 각 모듈을 순차적으로 실행
활용하려면 주어진 함수에서 **사용자정의 레이어(custom layer)** 를 쉽게 정의해야 함.
예를 들어, PyTorch에는 view 레이어가 없으므로 우리의 신경망 용으로 만들어야 함.
`Lambda` 는 `Sequential` 로 신경망을 정의할 때 사용할 수 있는 레이어를 생성


### GPU 사용하기
- 먼저 GPU가 PyTorch에서 작동하는지 확인
- `print(torch.cuda.is_available())`
- 디바이스 오브젝트를 생성
- `dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")`
- GPU로 배치를 옮기도록 `preprocess` 를 업데이트